# 1. 문자 단위 RNN(Char RNN)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

####  [apple을 입력받으면 pple!를 출력하는 RNN을 구현]

- **훈련 데이터 전처리하기**

In [2]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


In [3]:
# 입력의 크기는 문자 집합의 크기
input_size = vocab_size 
hidden_size = 5
output_size = 5
learning_rate = 0.1

# 문자에 고유한 정수 인덱스 부여
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) 
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [4]:
# 예측 결과를 다시 문자 시퀀스로 보기위해서 반대로 정수로부터 문자를 얻음
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [5]:
# 입력 데이터와 레이블 데이터의 각 문자들을 정수로 맵핑
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data) # a, p, p, l, e에 해당
print(y_data) # p, p, l, e, !에 해당

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [6]:
# 배치 차원 추가(파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력받음.)
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [7]:
# 입력 시퀀스의 각 문자들을 원-핫 벡터로 바꿔줌
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [8]:
# 입력 데이터와 레이블 데이터를 텐서로 바꿔줌
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

# 각 텐서의 크기를 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


- **모델 구현하기**

In [9]:
# RNN 모델을 구현
# fc는 완전 연결층(fully-connected layer)을 의미하며 출력층으로 사용
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        # RNN 셀 구현
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
        # 출력층 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) 
    
    # 구현한 RNN 셀과 출력층을 연결
    def forward(self, x): 
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [10]:
# 클래스로 정의한 모델을 net에 저장
net = Net(input_size, hidden_size, output_size)

# 모델에 입력을 넣어서 출력의 크기를 확인
outputs = net(X)
print(outputs.shape) # 3차원 텐서
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

torch.Size([1, 5, 5])
torch.Size([5, 5])


In [11]:
# 옵티마이저와 손실 함수를 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

# 총 100번의 에포크를 학습
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    # view를 하는 이유는 Batch 차원 제거를 위해
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) 
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result = outputs.data.numpy().argmax(axis=2) 
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.5619158744812012 prediction:  [[3 0 3 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  l!l!!
1 loss:  1.3192371129989624 prediction:  [[4 4 4 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppep
2 loss:  1.0482114553451538 prediction:  [[4 4 3 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplep
3 loss:  0.7714241743087769 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
4 loss:  0.5238173604011536 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
5 loss:  0.34458836913108826 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.22296974062919617 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.1414474993944168 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.09043611586093903 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.059101272374391556 prediction:  [[4 4 3 2

79 loss:  0.00031187976128421724 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
80 loss:  0.0003096874279435724 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
81 loss:  0.0003075666318181902 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
82 loss:  0.0003054696135222912 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
83 loss:  0.0003033964312635362 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
84 loss:  0.000301394728012383 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
85 loss:  0.00029936915962025523 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
86 loss:  0.00029741512844339013 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
87 loss:  0.0002954848459921777 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
88 loss:  0.00029

# 2. 문자 단위 RNN(Char RNN) - 더  많은 데이터

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

- **훈련 데이터 전처리하기**

In [13]:
# 임의의 샘플
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

# 중복을 제거한 문자 집합 생성
char_set = list(set(sentence)) 
# 각 문자에 정수 인코딩
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic) # 공백도 여기서는 하나의 원소

{'f': 0, 'k': 1, 'c': 2, "'": 3, 'w': 4, 'u': 5, 'l': 6, ',': 7, 'm': 8, 'b': 9, 'd': 10, '.': 11, 'n': 12, 'o': 13, 'y': 14, 'p': 15, 'h': 16, 'g': 17, 'r': 18, 't': 19, 'i': 20, 's': 21, 'e': 22, 'a': 23, ' ': 24}


In [14]:
# 문자 집합의 크기를 확인
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


In [15]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1

# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [16]:
# 첫번째 샘플의 입력 데이터와 레이블 데이터
print(x_data[0])
print(y_data[0])

[20, 0, 24, 14, 13, 5, 24, 4, 23, 12]
[0, 24, 14, 13, 5, 24, 4, 23, 12, 19]


In [17]:
# x 데이터 원-핫 인코딩
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

# 입력 데이터와 레이블 데이터를 텐서로 변환
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

# 훈련 데이터와 레이블 데이터의 크기를 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [18]:
# 원-핫 인코딩 된 결과를 보기 위해서 첫번째 샘플만 출력
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [19]:
# 레이블 데이터의 첫번째 샘플도 출력
print(Y[0])

tensor([ 0, 24, 14, 13,  5, 24,  4, 23, 12, 19])


- **모델 구현하기**

In [20]:
class Net(torch.nn.Module):
    # 현재 hidden_size는 dic_size와 같음.
    def __init__(self, input_dim, hidden_dim, layers): 
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [21]:
# 층을 두 개 쌓음
net = Net(dic_size, hidden_size, 2) 

# 비용 함수와 옵티마이저를 선언
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

# 모델에 입력을 넣어서 출력의 크기를 확인
outputs = net(X)
print(outputs.shape) # 3차원 텐서 <- (배치 차원 x 시점(timesteps) x 출력의 크기)

torch.Size([170, 10, 25])


In [22]:
# 정확도를 측정할 때는 이를 모두 펼쳐서 계산
# 배치 차원과 시점 차원을 하나로 만듦
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.

torch.Size([1700, 25])


In [23]:
for i in range(100):
    # 옵티마이저 정의
    optimizer.zero_grad()
    # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    outputs = net(X)
    # 손실 함수를 정의
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

eeeeeeeeweeeeeeeweeeeeeeeeweeeeweeeeeeeeeweweeeeweeeeeeeweeeeeeeeeweeeeeeeeeeeeeeeeeeewweeeeeeweeeewwweeeeeeeeeweeweeeweeweeeeweeeeeweeeeeeeeeeeeeeeeweeeeewwweeeeeeweeeeeeeeeweeww
ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
                                                                                                                                                                                   
t tthttt rttrtttrtttttrrrrtrtttrtrtrnttrtrtrttrhrrhtrntttttttthrrtnrttrtrrtrrrtrtttrttttrttrtrrttrtttrtt rtrtthrtttrttrrtrttttrhrrtnrttrtrhrrtnrtttrttttttnrtrtrttttttttthttttrttrt
tnttolto th tnttotnottntnnttttntnottnttootntnntoltn ttottnttntnotnnttnttntnnthtpnrttttn tnntn tnotnhnotnnttontnottnttontntnttsnhotnnttntnnnotnnltntoetntttn tnntnntttnnltnntoottott
ontootootootootootootodootoooodtoooontooontoodmotoodtmotmntoo ootodtootntoodootoodoodontoodtottodoot

p you want to build a ship, don't drum up people together to collect wood and don't dssign them tasks and work, but rather teach them to long for the endless immensity of the seac
p you want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the seac
p you want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the seac
p you want to build asship, don't drum up people together to collect wood and don't dssign them tasks and work, but rather teach them to long for the endless immensity of the seac
t you want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the seac
m you want to build a ship, don't arum up people together to collect wood and don't assign them task

m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't drum up people together to collect wood and don't assign them task

# 3. 단어 단위 RNN - 임베딩 사용

#### ['Repeat is the best medicine for'을 입력받으면 'is the best medicine for memory'를 출력하는 RNN]

- **훈련 데이터 전처리하기**

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim

In [25]:
# 임의의 문장
sentence = "Repeat is the best medicine for memory".split()

# 임의의 문장으로부터 단어장(vocabulary)을 만듦
vocab = list(set(sentence))
print(vocab)

['is', 'memory', 'Repeat', 'for', 'best', 'medicine', 'the']


In [26]:
# 단어장의 단어에 고유한 정수 인덱스를 부여
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
# 모르는 단어를 의미하는 UNK 토큰도 추가
word2index['<unk>']=0
print(word2index)

{'is': 1, 'memory': 2, 'Repeat': 3, 'for': 4, 'best': 5, 'medicine': 6, 'the': 7, '<unk>': 0}


In [27]:
# word2index에 단어를 입력하면 맵핑되는 정수를 리턴
print(word2index['memory'])

2


In [28]:
# 예측한 문장을 확인하기 위해 idx2word도 만듦
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'is', 2: 'memory', 3: 'Repeat', 4: 'for', 5: 'best', 6: 'medicine', 7: 'the', 0: '<unk>'}


In [29]:
# 데이터의 각 단어를 정수로 인코딩하는 동시에, 입력 데이터와 레이블 데이터를 만듦
def build_data(sentence, word2index):
    # 각 문자를 정수로 변환. 
    encoded = [word2index[token] for token in sentence]
    # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq, label_seq = encoded[:-1], encoded[1:] 
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [30]:
# 입력 데이터와 레이블 데이터
X, Y = build_data(sentence, word2index)
print(X)
print(Y)

tensor([[3, 1, 7, 5, 6, 4]])
tensor([[1, 7, 5, 6, 4, 2]])


- **모델 구현하기**

In [31]:
# 임베딩 층을 추가
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.
        self.linear = nn.Linear(hidden_size, vocab_size) 

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [32]:
# 하이퍼 파라미터
# 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
vocab_size = len(word2index)  
# 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
input_size = 5  
# RNN의 은닉층 크기
hidden_size = 20  

# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태
output = model(X)
print(output)

tensor([[ 0.2029,  0.0398, -0.0294, -0.0904,  0.1750,  0.2717, -0.0284, -0.2550],
        [ 0.1572,  0.0891, -0.1891, -0.1943,  0.0495,  0.3020, -0.1271, -0.3496],
        [ 0.0551, -0.0904, -0.4286, -0.0910,  0.4636,  0.3458, -0.0625, -0.1969],
        [ 0.2151, -0.0485, -0.1226, -0.1420,  0.2332,  0.3157,  0.0313, -0.0834],
        [ 0.4322, -0.0489,  0.1390,  0.0090, -0.0422,  0.0390, -0.2230, -0.1841],
        [ 0.3356,  0.0790,  0.1917,  0.0254, -0.1541,  0.0680, -0.2845, -0.4912]],
       grad_fn=<ViewBackward>)


In [33]:
# 예측값의 크기
print(output.shape)

torch.Size([6, 8])


In [34]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [35]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0722 
Repeat best best for best <unk> <unk>

[41/201] 1.5158 
Repeat is best best medicine for memory

[81/201] 0.8186 
Repeat is the best medicine for memory

[121/201] 0.3750 
Repeat is the best medicine for memory

[161/201] 0.1902 
Repeat is the best medicine for memory

[201/201] 0.1130 
Repeat is the best medicine for memory

